# Test the functions to make the prediction

In [ ]:
import pickle
import numpy as np 
import pandas as pd
import praw
from praw.models import MoreComments
import string
import nltk 
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import sys
import json

with open('pickles/tfidf.pkl', 'rb') as f:
    tfidf_ = pickle.load(f)

with open('pickles/best_rfc.pkl', 'rb') as f:
    predictor = pickle.load(f)

def ret_string(text):
    return str(text)

def remove_punc(text):
    text_nopunc = "".join([char for char in text if char not in string.punctuation])
    return text_nopunc

def tokenize(text):
    tokens = word_tokenize(text)
    tokens = [w.lower() for w in tokens]
    return tokens

def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    text = [word for word in text if word not in stop_words]
    return text

def lemmatizer(text):
    wn = nltk.WordNetLemmatizer()
    text = [wn.lemmatize(word) for word in text]
    return text

def predict_(text):
    text.strip()
    post_id = text.split('/')[-3]
    reddit = praw.Reddit(client_id='7hx8xkseiQkWNA', client_secret='UPd0tXn-Lt26mk4-Z38oCNLuG_8', user_agent='Kevin Stephen')
    post = reddit.submission(post_id)
    matrix = []
    comment = ''
    for top_level_comment in post.comments:
        if isinstance(top_level_comment, MoreComments):
            continue
        comment += top_level_comment.body 
    matrix.append([post.title, comment, post.url])
    test_df = pd.DataFrame(matrix, columns=['title', 'comment', 'url'])
    test_df['title'] = test_df['title'].apply(lambda x: ret_string(x))
    test_df['title'] = test_df['title'].apply(lambda x: remove_punc(x))
    test_df['title'] = test_df['title'].apply(lambda x: tokenize(x))
    test_df['title'] = test_df['title'].apply(lambda x: remove_stopwords(x))
    test_df['title'] = test_df['title'].apply(lambda x: lemmatizer(x))
    test_df['comment'] = test_df['comment'].apply(lambda x: ret_string(x))
    test_df['comment'] = test_df['comment'].apply(lambda x: remove_punc(x))
    test_df['comment'] = test_df['comment'].apply(lambda x: tokenize(x))
    test_df['comment'] = test_df['comment'].apply(lambda x: remove_stopwords(x))
    test_df['comment'] = test_df['comment'].apply(lambda x: lemmatizer(x))
    test_df['url'] = test_df['url'].apply(lambda x: ret_string(x))
    test_df['url'] = test_df['url'].apply(lambda x: remove_punc(x))
    test_df['url'] = test_df['url'].apply(lambda x: tokenize(x))
    test_df['url'] = test_df['url'].apply(lambda x: remove_stopwords(x))
    test_df['url'] = test_df['url'].apply(lambda x: lemmatizer(x))


    test_df['new_feature'] = test_df['title'] + test_df['comment'] + test_df['url']
    X_test = test_df['new_feature']
    test_df['new_feature']=[" ".join(word) for word in test_df['new_feature'].values]
    
    features_test = tfidf_.transform(X_test).toarray()
    pred = predictor.predict(features_test)
    
    with open('pickles/le_dict.pkl', 'rb') as f:
        le_dict_ = pickle.load(f)


    inv_dict = {v: k for k, v in le_dict_.items()} 

    print("The predicted flair is: {}".format(inv_dict[pred[0]]))


def predict_multiple(path):
    matrix = []
    urls = []
    with open(path, 'r+') as f:
        text = f.readline().strip()
        while text:
            post_id = text.split('/')[-3]
            reddit = praw.Reddit(client_id='7hx8xkseiQkWNA', client_secret='UPd0tXn-Lt26mk4-Z38oCNLuG_8', user_agent='Kevin Stephen')
            post = reddit.submission(post_id)
            comment = ''
            for top_level_comment in post.comments:
                if isinstance(top_level_comment, MoreComments):
                    continue
                comment += top_level_comment.body 
            matrix.append([post.title, comment, post.url])
            urls.append(text)
            text = f.readline().strip()
    urls_df = pd.DataFrame(urls, columns=['post_url'])
    test_df = pd.DataFrame(matrix, columns=['title', 'comment', 'url'])
    test_df['title'] = test_df['title'].apply(lambda x: ret_string(x))
    test_df['title'] = test_df['title'].apply(lambda x: remove_punc(x))
    test_df['title'] = test_df['title'].apply(lambda x: tokenize(x))
    test_df['title'] = test_df['title'].apply(lambda x: remove_stopwords(x))
    test_df['title'] = test_df['title'].apply(lambda x: lemmatizer(x))
    test_df['comment'] = test_df['comment'].apply(lambda x: ret_string(x))
    test_df['comment'] = test_df['comment'].apply(lambda x: remove_punc(x))
    test_df['comment'] = test_df['comment'].apply(lambda x: tokenize(x))
    test_df['comment'] = test_df['comment'].apply(lambda x: remove_stopwords(x))
    test_df['comment'] = test_df['comment'].apply(lambda x: lemmatizer(x))
    test_df['url'] = test_df['url'].apply(lambda x: ret_string(x))
    test_df['url'] = test_df['url'].apply(lambda x: remove_punc(x))
    test_df['url'] = test_df['url'].apply(lambda x: tokenize(x))
    test_df['url'] = test_df['url'].apply(lambda x: remove_stopwords(x))
    test_df['url'] = test_df['url'].apply(lambda x: lemmatizer(x))
    test_df['new_feature'] = test_df['title'] + test_df['comment'] + test_df['url']
    
    X_test = test_df['new_feature']
    test_df['new_feature']=[" ".join(word) for word in test_df['new_feature'].values]

    features = tfidf_.transform(X_test).toarray() 
    pred = predictor.predict(features)
    pred_df = pd.DataFrame(pred, columns=['prediction'])
    json_df = pd.concat([urls_df, pred_df], axis=1)
    result_dict = dict(zip(json_df.post_url, json_df.prediction))
    json_result = json.dumps(result_dict)

    with open('predictions.json', 'w+') as f:
        json.dump(json_result, f)

    
if __name__ == "__main__":
    # url = sys.argv[1]
    predict_multiple('./urls.txt')
